In [1]:
import sys
import os
import collections
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import re
from tabula import read_pdf
import csv

import requests
from bs4 import BeautifulSoup

In [2]:
%pip install google-search-results
from serpapi import GoogleSearch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [42]:
def calculate_scores(user_data):

    data_bac = pd.read_csv("./BAC CSV/bac2019.csv")
    # Automatically map subjects to their respective fields
    subjects = {
        user_data.get('Subiect ea', ''): user_data.get('NOTA_FINALA_EA', 0),
        user_data.get('Subiect ec', ''): user_data.get('NOTA_FINALA_EC', 0),
        user_data.get('Subiect ed', ''): user_data.get('NOTA_FINALA_ED', 0)
    }

    # Extract other relevant fields
    digital_skills = user_data.get('PUNCTAJ DIGITALE', 0)
    communication_skills = user_data.get('ORAL_PMO', 0)
    overall_grade = user_data.get('Medie', 0)
    profile = user_data.get('Profil', '')
    specialization = user_data.get('Specializare', '')

    # Initialize subject-based variables
    math_grade = 0
    science_grade = 0
    language_grade = 0
    history_grade = 0
    art_grade = 0
    sports_grade = 0
    vocational_grade = 0
    logic_grade= 0
    info_grade = 0

    # Assign grades based on subject mapping
    for subject, grade in subjects.items():
        if any(math_subj in subject for math_subj in ['Matematică MATE-INFO', 'Matematică ST-NAT', 'Matematică TEHN']):
            math_grade = grade
        elif any(sci in subject for sci in ['Biologie vegetală și animală', 'Anatomie și fiziologie umană, genetică și ecologie umană', 'Chimie organică TEH Nivel I/II', 'Chimie anorganică TEH Nivel I/II', 'Chimie organică TEO Nivel I/II', 'Chimie anorganică TEO Nivel I/II', 'Fizică TEO', 'Fizică TEH']):
            science_grade = grade
        elif any(lang in subject for lang in ['Limba română (UMAN)', 'Limba română (REAL)']):
            language_grade = grade
        elif any(hist in subject for hist in ['Istorie', 'Geografie']):
            history_grade = grade
        elif 'Logică, argumentare și comunicare' in subject:
            logic_grade = grade
        elif 'I' in subject:
            info_grade = grade
            
    # Penalty for low overall grade
    difficulty_penalty = 1 if overall_grade >= 8 else 0.8 if overall_grade >= 6 else 0.5

    # Profile-based penalty or boost
    profile_adjustments = {
        'Uman': {'Computer Science': 0.2, 'Technical': 0.3, 'Medicine': 0.4, 'Psychology': 1.2, 'Political Science': 1.2, 'Journalism': 1.3, 'Arts': 1.4},
        'Real': {'Law': 0.4, 'Psychology': 0.5, 'Arts': 0.3, 'Computer Science': 1.2},
        'Tehnologică': {'Medicine': 0.5, 'Law': 0.6, 'Arts': 0.4, 'Technical' : 1.2},
        'Vocatională': {'Engineering': 0.5, 'Computer Science': 0.5, 'Arts': 1.2}
    }

    profile_penalty = profile_adjustments.get(profile, {})

    # Scores for each field with penalty applied
    scores = {}

    # Architecture
    scores['Architecture'] = difficulty_penalty * (0.4 * math_grade + 0.002 * digital_skills + 0.4 * overall_grade) * profile_penalty.get('Architecture', 1)


    # Law
    if logic_grade > 6:
        scores['Law'] = difficulty_penalty * (0.4 * language_grade + 0.2 * history_grade + 0.1 * overall_grade + 0.1 * communication_skills + 0.2 * logic_grade) * profile_penalty.get('Law', 1)
    else: scores['Law'] = difficulty_penalty * (0.4 * language_grade + 0.4 * history_grade + 0.1 * overall_grade + 0.1 * communication_skills) * profile_penalty.get('Law', 1)

    # Geography
    scores['Geography'] = difficulty_penalty * (0.5 * history_grade + 0.3 * communication_skills + 0.2 * overall_grade)
    
    # Journalism
    scores['Journalism'] = difficulty_penalty * (0.5 * language_grade + 0.4 * communication_skills + 0.1 * overall_grade) * profile_penalty.get('Journalism', 1)

    # Psychology
    scores['Psychology'] = difficulty_penalty * (0.3 * science_grade + 0.3 * language_grade + 0.2 * overall_grade + 0.2 * communication_skills) * profile_penalty.get('Psychology', 1)

    # Political Science (SNSPA)
    scores['Political Science'] = difficulty_penalty * (0.5 * language_grade + 0.3 * history_grade + 0.2 * overall_grade) * profile_penalty.get('Political Science', 1)
    
    # Medicine
    scores['Medicine'] = difficulty_penalty * (0.6 * science_grade + 0.3 * overall_grade + 0.001 * digital_skills) * profile_penalty.get('Medicine', 1)

    # Computer Science
    if info_grade > 5:
        scores['Computer Science'] = difficulty_penalty * (0.3 * math_grade + 0.4 * info_grade + 0.002 * digital_skills + 0.1 * overall_grade) * profile_penalty.get('Computer Science', 1)
    else: scores['Computer Science'] = difficulty_penalty * (0.6 * math_grade + 0.003 * digital_skills + 0.1 * overall_grade) * profile_penalty.get('Computer Science', 1)    

    # Mathematics
    scores['Mathematics'] = difficulty_penalty * (0.7 * math_grade + 0.2 * overall_grade + 0.1 * science_grade)

    # Physical Education & Sports
    if profile == 'Educație fizică și sport':
        scores['Physical Education'] = difficulty_penalty * (0.5 * 10 + 0.3 * overall_grade + 0.2 * communication_skills)
    else: scores['Physical Education'] = difficulty_penalty * (0.6 * sports_grade + 0.2 * overall_grade + 0.2 * communication_skills)

    # Technical Fields
    scores['Technical'] = difficulty_penalty * (0.5 * math_grade + 0.3 * science_grade + 0.002 * digital_skills) * profile_penalty.get('Technical', 1)

    # Arts
    if profile == 'Artistic':
        scores['Arts'] = difficulty_penalty * (0.5 * 10 + 0.3 * language_grade + 0.2 * overall_grade) * profile_penalty.get('Arts', 1)
    else: scores['Arts'] = difficulty_penalty * (0.6 * language_grade + 0.3 * communication_skills + 0.1 * overall_grade) * profile_penalty.get('Arts', 1)

    # Recommend the best field
    recommended_field = max(scores, key=scores.get)
    
    return recommended_field, scores

In [45]:
user_data = {
    'Subiect ea': 'Limba română (UMAN)',           # Limba si literatura romana
    'Subiect ec': 'Geografie',                     # Disciplina obligatorie profil
    'Subiect ed': 'Biologie vegetală și animală',  # Disciplina la alegere
    'NOTA_FINALA_EA': 5,
    'NOTA_FINALA_EC': 5,
    'NOTA_FINALA_ED': 5,
    'PUNCTAJ DIGITALE': 55,
    'ORAL_PMO': 5,
    'Medie': 5,
    'Profil': 'Uman',
    'Specializare': 'Filologie'
}

user_data2 = {
    'Subiect ea': 'Limba română (REAL)',
    'Subiect ec': 'Matematică MATE-INFO',
    'Subiect ed': 'Fizică TEO',
    'NOTA_FINALA_EA': 6.01,
    'NOTA_FINALA_EC': 5.25,
    'NOTA_FINALA_ED': 8.5,
    'PUNCTAJ DIGITALE': 22.0,
    'ORAL_PMO': 5.25,
    'Medie': 6.01,
    'Profil': 'Real',
    'Specializare': 'Matematica-Informatica'
}


user_data3 = {
    'Subiect ea': 'Limba română (REAL)',
    'Subiect ec': 'Matematică MATE-INFO',
    'Subiect ed': 'Informatică MI C/C++',
    'NOTA_FINALA_EA': 8.01,
    'NOTA_FINALA_EC': 9.25,
    'NOTA_FINALA_ED': 10,
    'PUNCTAJ DIGITALE': 22.0,
    'ORAL_PMO': 8.25,
    'Medie': 9.08,
    'Profil': 'Real',
    'Specializare': 'Matematica-Informatica'
}

In [46]:
recommendation, all_scores = calculate_scores(user_data)
print(f"Recommended Field of Study: {recommendation}")
print("Detailed Scores:")
for field, score in sorted(all_scores.items(), key=lambda item: item[1], reverse=True):
    print(f"{field}: {score:.2f}")
print()

recommendation, all_scores = calculate_scores(user_data2)
print(f"Recommended Field of Study: {recommendation}")
print("Detailed Scores:")
for field, score in sorted(all_scores.items(), key=lambda item: item[1], reverse=True):
    print(f"{field}: {score:.2f}")
print()

recommendation, all_scores = calculate_scores(user_data3)
print(f"Recommended Field of Study: {recommendation}")
print("Detailed Scores:")
for field, score in sorted(all_scores.items(), key=lambda item: item[1], reverse=True):
    print(f"{field}: {score:.2f}")
print()

Recommended Field of Study: Arts
Detailed Scores:
Arts: 3.50
Journalism: 3.25
Psychology: 3.00
Political Science: 3.00
Law: 2.50
Geography: 2.50
Architecture: 1.05
Physical Education: 1.00
Medicine: 0.91
Mathematics: 0.75
Technical: 0.24
Computer Science: 0.07

Recommended Field of Study: Medicine
Detailed Scores:
Medicine: 5.54
Mathematics: 4.58
Journalism: 4.56
Technical: 4.18
Computer Science: 3.66
Architecture: 3.64
Political Science: 3.37
Psychology: 2.64
Geography: 2.22
Physical Education: 1.80
Arts: 1.39
Law: 1.13

Recommended Field of Study: Computer Science
Detailed Scores:
Computer Science: 9.27
Mathematics: 8.29
Journalism: 8.21
Architecture: 7.38
Political Science: 5.82
Technical: 4.67
Geography: 4.29
Physical Education: 3.47
Psychology: 2.93
Medicine: 2.75
Arts: 2.46
Law: 1.97



In [7]:
from serpapi import GoogleSearch

def search_universities_near_city(city, interest):
    params = {
        "engine": "google",
        "q": f"universities near {city} Romania {interest}",
        "location": f"{city}, Romania",
        "num": 10,
        "api_key": "9e0409085db46c894a9646b2d03041e2954fc586212772d70c2ab195e435dd69"
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    
    universities = []
    
    # Expanded list of related keywords
    keywords = [
        "university", "universitate", "faculty", "facultate", 
        "institute", "institut", "college", "colegiu", 
        "academia", "academie", "polytechnic", "politehnică", 
        "school of", "școală", "univ", "institute of"
    ]

    for result in results.get('organic_results', []):
        title = result.get('title', '').lower()
        link = result.get('link')

        # Check if any keyword is present in the title
        if any(keyword in title for keyword in keywords):
            universities.append({'Title': result.get('title'), 'Link': link})

    return universities

In [8]:
# Example usage
filtered_results = search_universities_near_city("Iasi", "medicina")

# Print the filtered results
for uni in filtered_results:
    print(f"{uni['Title']}: {uni['Link']}")

University of Medicine and Pharmacy "Grigore T. Popa": https://www.umfiasi.eu/en
University of Medicine Iasi: https://www.studyinginromania.com/university-of-medicine-iasi.html
Faculty of Medicine: https://www.umfiasi.ro/en/academic/facultati/medicina-generala/
Gr. T. Popa University of Medicine in Iasi, Romania: https://europe-studies.com/en/medicine-university-iasi-romania/
University of Medicine and Pharmacy Grigore T Popa Iasi: https://www.usnews.com/education/best-global-universities/university-of-medicine-and-pharmacy-grigore-t-popa-iasi-529245
6 Universities in Iasi | Rankings & Ratings 2025: https://www.universityguru.com/universities-iasi
University of Medicine and Pharmacy Iasi: https://apply.eu-medstudy.com/iasi
University of Medicine and Pharmacy Grigore T. Popa, Iasi ...: https://www.mastersportal.com/universities/18616/university-of-medicine-and-pharmacy-grigore-t-popa-iasi-romania.html
Grigore T. Popa University of Medicine: https://www.medlinkstudents.com/universities/i